### Finite difference modeling of stream power law river profile evolution

In this notebook, we will progressively build the components of a 1-D longitudinal bedrock river profile model by solving the stream power law river profile evolution equation using a finite difference approximation. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

---
#### Exercise 1: Discretizing the river profile
Let’s consider a river, initiating at a channel head 200 m from the ridgeline and flowing 4800 m downstream to its outlet. We’ll establish a coordinate system with our x-coordinate denoting the streamwise distance from the ridgeline, with the channel head at distance ${x_c}$ = 200 m and the outlet at distance ${x_L}$ = 5000 m

Let’s start off by defining our streamwise distance coordinates at evenly spaced increments ${dx}$ = 10 m. 

| Parameter | Value |
| --- | --- |
| x<sub>c</sub> | 200 m |
| x<sub>L</sub> | 5000 m |
| dx | 10 m|

**1.** Use `np.arange()` or `np.linspace()` to create an array of ${x}$ distance coordinates. Consider running e.g. `np.arange?` or `help(np.arange)` to check the documentation of the function - particularly how it handles the ends of the interval. 

**2.** Use `%whos` or `len()` to check the length of the resulting array and `np.min()` and `np.max()` to check its range.

---

---
#### Exercise 2: Drainage area along the river profile
The amount of river discharge likely affects the rate of bedrock river incision at each point along the river profile. Measuring discharge continuously at each point along a river would require a lot of stream gauges! Fortunately, **drainage area** (the planview area of a river basin contributing water flow to a given point on the landscape) provides a reasonable proxy for discharge in many landscapes and can easily be computed from a digital elevation model (DEM) or landscape evolution model using a flow routing algorithm (you'll write your own code to do this later in the course). Because, for now, we are only considering a 1-D river profile, we'll use an empirical relationship between distance from the ridgeline ${x}$ and drainage area ${A(x)}$, **Hack's Law**, to assign the drainage area at each distance node ([Hack 1957](https://pubs.usgs.gov/pp/0294b/report.pdf)):

$${A = k_a x^h}$$

**1.**  Let's calculate drainage area ${A(x)}$ along our river profile using Hack's Law with:

| Parameter | Value |
| --- | --- |
| k<sub>a</sub> | 0.5 m<sup>(2-h)<sup> |
| h | 2|

**2.**  Plot drainage area vs. streamwise distance from the ridgeline
    
---

---
#### Exercise 3: Steady-state river profile
We found an analytic solution for the steady-state (${\frac{\partial z}{\partial t} = 0}$) elevations of a river profile using the stream power law (SPL) river profile evolution equation, ${\frac{\partial z}{\partial t} = U - KA^m{\left|\frac{\partial z}{\partial x}\right|}^{n}}$:

$${z_{ss}(x) =\begin{cases}
  z_L - ({\frac{U}{K {k_a}^m}})^{\frac{1}{n}}\frac{n}{n-hm}(x^{\frac{n-hm}{n}} - {x_L}^{\frac{n-hm}{n}}) & n \neq hm \\
  z_L - ({\frac{U}{K {k_a}^m}})^{\frac{1}{n}}\rm{ln}(\frac{x}{x_L}) & n = hm
\end{cases}}$$

**1.** Write a function to calculate the steady-state river profile for any given distance array, outlet elevation, uplift rate, and Hack's Law and SPL parameters

**2.** Use your function to calculate the steady-state river profile for the parameters above and...

| Parameter | Value |
| --- | --- |
| z<sub>L</sub> | 0 m|
| U | 0.001 m/yr |
| K | 1e-5 yr<sup>-1</sup> |
| m | 0.5 |
| n | 1 |


**3.** Plot elevation vs. streamwise distance from the ridgeline

---

---
#### Exercise 4: Finite difference approximation of the spatial derivative
We've now created an array of steady-state river profile elevations ${z_{ss}}$ values. Let's consider any such arbitrary elevation array:

`z[0]` `z[1]` `z[2]` `...` `z[-3]` `z[-2]` `z[-1]`

Headwater \-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\> Outlet

...with distance spacing between grid points ${dx}$. We have three simple options (backwards, forward/upwind, and centered differencing) to calculate the finite difference approximation of the slope ${\frac{dz}{dx}}$ at each grid point. E.g.

**Forward/upwind:** ${\frac{dz}{dx} \approx \frac{z(x+ \Delta x) - z(x)}{\Delta x}}$ **Centered:** ${\frac{dz}{dx} \approx \frac{z(x + \Delta x) - z(x - \Delta x)}{2\Delta x}}$ 

Taking forward space differencing as an example, we can approximate the slope at any given grid point ${z(x)}$ by subtracting its downstream neighbor ${z(x + \Delta x)}$ from it and dividing by the grid step ${dx}$. We can do this at every grid point except the outlet grid point `z[-1]`, which has no downstream neighbor ${z(x + \Delta x)}$. For consistency and later calculations, we'll want to store the slope we calculate for each grid point at the same index in a new slope array. So, we first want to preallocate an output array the same size as our z array to store the slope at each node using the function `np.zeros()`. We can't calculate the forward space slope at `z[-1]` because the outlet has no downstream neighbor, so we want to end up with... 

 `S[0]` `S[1]` `...` `S[-3]` `S[-2]` `   0   `

We could loop through this array with a `for` loop to compute each slope, e.g.

    for i in range(0,len(z)):
        S[i] = (z[i] - z[i+1])/dx

...but there's a more efficient way to do this in python using array indexing. Recall that we can access all but the last ${i}$ elements of an array using the syntax `z[:-i]` or all but the first ${i}$ elements using the syntax `z[i:]`. 

**1.** Using array indexing, write a function that returns the forward and centered space finite difference approximation of the slope of an input elevation array ${z}$ with distance spacing ${dx}$.


**2.** Plot the slope you calculate for each method for the steady-state river profile vs. streamwise distance from the ridgeline

---

---
#### Exercise 5: River profile update
To model the transient evolution of a bedrock river profile or solve numerically for its steady-state form, we also need to discretize the evolution equation: 

$${\frac{\partial z}{\partial t} = U - KA^{m} S^{n}}$$  

...in time. We'll first consider forward time difference schemes, where we approximate the time derivative:

$${\frac{\partial z}{\partial t} \approx \frac{z(t + \Delta t) - z(t)}{\Delta t}}$$

So...

$${z(t + \Delta t) = z(t) + \Delta t (U - KA^{m} S^{n})}$$

Recall we already have a function that approximates the spatial derivative ${S}$ using a forward/upwind or centered space difference scheme. We'll also need to specify an **initial condition** for the river profile and **boundary conditions** at the outlet (and at the channel head too if we use centered space differencing). There are two main types of boundary conditions:

- **Dirichlet/fixed:** Elevation ${z}$ is specified at the outlet (or channel head)
- **Neumann:** Channel slope ${\frac{\partial z}{\partial x}}$ is specified at the outlet (or channel head)

**1.** Write a function that updates river profile elevations using forward time upwind space (FTUS) and forward time centered space (FTCS) finite differencing over one time step. Your function should allow you to specify an initial condition and different types of boundary conditions.

**2.** Check that your functions returns returns reasonable output for the first time step, assuming an initial straight slope decreasing from an elevation of 500 m at the channel head to 0 m at the outlet, the parameters above, and...

| Parameter | Value |
| --- | --- |
| dt| 100 yr|

using... 
- FTUS and a Dirichlet outlet boundary condition with z<sub>out</sub> = 0 m
- FTUS and a Neumann outlet boundary condition with ${\frac{\partial z}{\partial x}}$ = 0 m/m
- FTCS and a Dirichlet outlet boundary condition with z<sub>out</sub> = 0 m
- FTCS and a Neumann outlet boundary condition with ${\frac{\partial z}{\partial x}}$ = 0 m/m

…and, if necessary, a Neumann headwater boundary condition that maintains the upwind slope of the initial profile at the channel head (you can write this directly into the river profile evolution update function)

---

---
#### Exercise 6: River profile evolution

We now have a function that will update a river profile using the SPL evolution equation over a specified time step. In order to  model the longer-term evolution of the river profile, for instance over a duration ${T}$, all we need to do is progressively update the river profile at time steps ${dt}$. 

Whereas we could use array indexing (and avoid using a loop) to calculate channel slopes, here we would like to **iteratively** call our update function (i.e. apply the function repeatedly, using the output from one iteration as the input to the next)

`zn = f(z0,...)` → `zn = f(zn,...)` → `zn = f(zn,...)` → ... → `zn = f(zn,...)`

We would like to do this at every time step ${dt}$ over a duration ${T}$, so we could use either a `for` loop or a `while` loop to advance in time.

**1.** Write a function that models the transient evolution of a bedrock river profile using a specified finite differencing scheme, time step, model duration, and stream power and Hack's law parameters.

**2.** Using the same model/parameter values, the initial straight slope, a Dirichlet outlet B.C. with z<sub>out</sub> = 0 m, and a Neumann headwater B.C. that maintains the initial slope, plot the river profiles every 100 kyr for 500 kyr using FTUS and FTCS differencing.

---
#### Exercise 7: CFL condition for the SPL evolution equation 

Methods used to solve wave equations are typically unstable if the information in the solution propagates a distance longer than one grid point (Δx) in one time step (Δt). To ensure that the solution propagates no more than one grid point in one time step, we must satisfy the **Courant-Friedrichs-Lewy (CFL) condition**:

$${C =\frac{u \Delta t}{\Delta x} \leq 1}$$

For the stream power bedrock river evolution equation:

$${u = K A^m S^{n-1}}$$

Recall that stability requires that *every* point satisfies the CFL condition, so your choice of Δt will be limited by the point along the profile at which ${u}$ is largest. If ${n}$=1 and we use Hack's Law, ${u}$ only depends on the streamwise distance along the profile (and will not change over time), whereas if ${n}$≠1, ${u}$ depends on channel slope, which evolves with uplift and erosion over time.

**1.** Write a function to compute the maximum allowable time step satisfying the CFL condition for a specified river profile and Hack's law and stream power parameters. 

**2.** Use this function to calculate the maximum allowable time step using our default model parameters. What portion of the profile limits the time step? How does the maximum time step change if we vary ${K}$, ${n}$, or ${dx}$?

**3.** Now try running the same river profile evolution model using this maximum allowable time step and also a time step 100 yrs longer. Copy your code from Exercise 6 to model and plot river profiles every 100 kyr for 500 kyr using FTUS.

---

---
#### Exercise 8: Numerical vs. analytic steady-state river profiles 

How do we know if our solution is accurate? Well, if analytic solutions exist, we can compare our numerical solutions to the exact solution at each discretized grid point. We already found an analytic solution for the steady-state form of the stream power evolution equation. If we run our numerical model until it also reaches a steady state, we can then compare the analytic and numerical solution directly to quantify the error.

**Mean absolute error:** ${\rm{mean}(|z_{mod,i} - z_i|)}$

**Maximum absolute error:** ${\rm{max}(|z_{mod,i} - z_i|)}$

**Root mean squared error:** ${\frac{1}{n}\Sigma_{i=1}^n(z_{mod,i}-z_i)^2}$

**Normalized root mean squared error:** ${\frac{\frac{1}{n}\Sigma_{i=1}^n(z_{mod,i}-z_i)^2}{\rm{max}(z_i) - \rm{min}(z_i)}}$

**1.** Modify your river profile evolution function to print out a statement if and when we reach a steady-state, which we'll define as the point at which the rate of river profile elevation change does not exceed 1e-6 m/yr at any point along the profile. How long does it take for our default case to reach a steady state?

**2.** Now let's compare our numerical and analytic steady-state river profiles for the default case. Plot both profiles and compute the mean and maximum absolute error and the root mean squared error between them. Try this also using dx = 20 m.  

---